In [7]:
import sys
sys.path.append('../')
from SQLConnection import LocalConnection
from SQLConnection import AWSConnection
sys.path.append('../databases')
import mysql_aws_credentials as aws
from Format import Format
import re

In [8]:
local_conn = LocalConnection('../databases/main.db')

In [9]:
aws_conn = AWSConnection(aws.HOST, aws.PORT, aws.DATABASE_NAME, aws.USER, aws.PASSWORD)

In [ ]:
cons_test_users = aws_conn.get_users_by_pol_label('conservative')
libs_test_users = aws_conn.get_users_by_pol_label('liberal')
cons_train_users = local_conn.get_users_by_pol_label('conservative')
libs_train_users = local_conn.get_users_by_pol_label('liberal')

In [ ]:
print(cons_train_users.__len__())
print(libs_train_users.__len__())
cons_train_users[12]

In [ ]:
test_tweets = local_conn.get_all_tweets_by('sentedcruz')
test_tweets1 = local_conn.get_all_tweets_by('aoc')

In [ ]:
test_tweet = test_tweets[0][1]

In [1]:
import nltk
nltk.download('stopwords')
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/knownastron/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import nltk
nltk.download('stopwords')
import string
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(input_text):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    stopwords.add('it\'s')
    stopwords.add('w/')
    stopwords.add('\'s')
    input_text_split = input_text.split()
    output_text_split = [word for word in input_text_split if word not in stopwords]
    return " ".join(output_text_split)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/knownastron/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def denoise_tweet(tweet_text):    
    tweet_text = tweet_text.lower()
    tweet_text = Format.remove_hyperlinks(tweet_text)
    tweet_text = Format.remove_picture_links(tweet_text)
    tweet_text = Format.remove_hashtags(tweet_text)
    tweet_text = Format.remove_mentions(tweet_text)
    tweet_text = Format.remove_picture_links(tweet_text)
    tweet_text = remove_stopwords(tweet_text)
    tweet_text = tweet_text.translate(str.maketrans('','',string.punctuation + '—' + '“' + '…' + '\’')) #remove punctuation
    return tweet_text
    

In [ ]:
denoised = []
denoised1 = []
for tweet in test_tweets:
    denoised.append(denoise_tweet(tweet[1]))
    
for tweet in test_tweets1:
    denoised1.append(denoise_tweet(tweet[1]))

In [ ]:
tokens = []
for i,d in enumerate(denoised):
    tokens.extend(d.split())
    
tokens1 = []
for i,d in enumerate(denoised1):
    print(i, test_tweets1[i][1])
    print('////')
    print(d,'\n')
    tokens1.extend(d.split())

In [ ]:
import collections
count = collections.Counter(tokens)
# count.most_common(50)

In [ ]:
import collections
count1 = collections.Counter(tokens1)
# count1.most_common(50)

In [ ]:
print(t, '\n')
t = Format.remove_hashtags(t)
print(t, '\n')
t = Format.remove_mentions(t)
print(t, '\n')
t = remove_stopwords(t)
print(t, '\n')
t = Format.remove_picture_links(t)
print(t, '\n')
t = t.translate(str.maketrans('','',string.punctuation + '—' + '“' + '…'))
print(t, '\n')
t.split()

In [ ]:
tweet_text = test_tweets[73][1]
tweet_text

In [ ]:
tweet_text = Format.remove_picture_links(tweet_text)
print(tweet_text, '\n')
tweet_text = tweet_text.lower()
print(tweet_text, '\n')
tweet_text = Format.remove_hashtags(tweet_text)
print(tweet_text, '\n')
tweet_text = Format.remove_mentions(tweet_text)
print(tweet_text, '\n')


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import classification_report
from sklearn import svm 
from sklearn.model_selection import train_test_split

In [100]:
cons_test_users = aws_conn.get_users_by_pol_label('conservative')
libs_test_users = aws_conn.get_users_by_pol_label('liberal')
cons_train_users = local_conn.get_users_by_pol_label('conservative')
libs_train_users = local_conn.get_users_by_pol_label('liberal')

In [102]:
print(len(cons_test_users))
print(len(libs_test_users))
print(len(cons_train_users))
print(len(libs_train_users))

51
29
258
291


In [62]:
# ('sentedcruz', 1074480192, '1074480192', 'conservative', None)
corpus = []
for i, user in enumerate(cons_train_users):
    tweets = local_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = denoise_tweet(tweets)
    corpus.append((user[0],tweets, user[3]))
    
for user in libs_train_users:
    tweets = local_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = denoise_tweet(tweets)
    corpus.append((user[0], tweets, user[3]))

In [63]:
for i, user in enumerate(cons_test_users):
    tweets = local_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = denoise_tweet(tweets)
    corpus.append((user[0], tweets, user[3]))
    
for i, user in enumerate(libs_test_users):
    tweets = local_conn.get_all_tweets_by(user[0])
    tweets = [x[1] for x in tweets]
    tweets = " ".join(tweets)
    tweets = denoise_tweet(tweets)
    corpus.append((user[0], tweets, user[3]))

In [78]:
len(corpus)


629

In [79]:
data = []
labels = []
for row in corpus:
    data.append(row[1])
    labels.append((row[0], row[2]))

In [164]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.5, random_state=6)

In [165]:
y_train_username = []
y_train_label = []

for y in y_train:
    y_train_username.append(y[0])
    y_train_label.append(y[1])

In [166]:
print(len(y_train_username))
print(len(y_train_label))

314
314


In [167]:
y_test_username = []
y_test_label = []

for y in y_test:
    y_test_username.append(y[0])
    y_test_label.append(y[1])

In [168]:
print(len(y_test_username))
print(len(y_test_label))

315
315


In [169]:
# data = test_data + train_data
# labels = test_labels + train_labels
# x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.33, random_state=42)

In [170]:
# Create feature vectors 
vectorizer = TfidfVectorizer(min_df=4, max_df=0.9)
# Train the feature vectors
train_vectors = vectorizer.fit_transform(x_train) #(train_data)
# Apply model on test data 
test_vectors = vectorizer.transform(x_test)#(test_data)

In [171]:
# Perform classification with SVM, kernel=linear 
model = svm.SVC(kernel='linear') 
model.fit(train_vectors, y_train_label) 
prediction = model.predict(test_vectors)


In [172]:
print(len(prediction))

315


In [173]:
for i, pred in enumerate(prediction):
    if y_test_label[i] != pred:
        print(y_test_username[i], pred)

wonkroom conservative
jbarro liberal
senatortester conservative
sanfordbishop conservative
staceyplaskett conservative
mindyfinn liberal
repbenmcadams conservative
repronkind conservative
obsoletedogma conservative
chrislynnhedges conservative
senamyklobuchar conservative
repedcase conservative
repgolden conservative
tamikadmallory conservative
senatemajldr conservative
senatorsinema conservative
repcindyaxne conservative
chrismurphyct conservative
ezraklein conservative
bobbyscott conservative
kairyssdal conservative
heerjeet conservative
deray conservative
julietlapidos conservative
daveloebsack conservative
repbrianfitz liberal
neonflag conservative
mariodb liberal
collinpeterson conservative


In [174]:
print (classification_report(y_test_label, prediction))
# print(len(y_test_label))

              precision    recall  f1-score   support

conservative       0.86      0.97      0.91       154
     liberal       0.97      0.84      0.90       161

    accuracy                           0.91       315
   macro avg       0.91      0.91      0.91       315
weighted avg       0.92      0.91      0.91       315

